In [1]:
import numpy as np
training_data = np.load('/datasets/home/32/032/sugao/CSE190/pa4/train.npy')

In [2]:
from dataloader import *
from models import *
#from configs import cfg
import time

cfg = {}
cfg['input_dim'] = 204 # input dimension to LSTM
cfg['hidden_dim'] = 128 # hidden dimension for LSTM
cfg['output_dim'] = 99 # output dimension of the model
cfg['layers'] = 2 # number of layers of LSTM
cfg['dropout'] = 0.005 # dropout rate between two layers of LSTM; useful only when layers > 1; between 0 and 1
cfg['bidirectional'] = False # True or False; True means using a bidirectional LSTM
cfg['batch_size'] = 150 # batch size of input
cfg['learning_rate'] = 5e-4 # learning rate to be used
cfg['L2_penalty'] = 1e-5 # weighting constant for L2 regularization term; this is a parameter when you define optimizer
cfg['gen_temp'] = 0.01 # temperature to use while generating reviews
cfg['max_len'] = 2000 # maximum character length of the generated reviews
cfg['epochs'] = 6 # number of epochs for which the model is trained
cfg['cuda'] = True #True or False depending whether you want to run your model on a GPU or not. If you set this to True, make sure to start a GPU pod on ieng6 server
cfg['train'] = True # True or False; True denotes that the model is bein deployed in training mode, False means the model is not being used to generate reviews
cfg['train_split'] = 0.8
cfg['device'] = 'cuda'

model = BeerLstmMind(cfg, True)
model = model.to(torch.device(cfg['device']))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=cfg['learning_rate'], weight_decay=cfg['L2_penalty'])
train_indices, valid_indices = train_valid_split(len(training_data), cfg['train_split'])
np.save('val2.npy', valid_indices)
print('ready')

ready


In [3]:
 for epoch in range(cfg['epochs']):
    tloader = DataLoader(training_data, train_indices, cfg)
    vloader = DataLoader(training_data, valid_indices, cfg)
    
    count = 0
    avg_loss = 0
    while tloader.has_next():
        #start = time.perf_counter()
        train, label = tloader.get_next()
        #end = time.perf_counter()
        #print('get train & label', end - start)
        
        model.zero_grad()
        
        #start = time.perf_counter()
        y = model(train)
        #end = time.perf_counter()
        #print('train', end - start)
        
        y = y.permute(0, 2, 1)
        
        loss = criterion(y, label)
        loss.backward()
        optimizer.step()

        count += 1
        avg_loss += loss.item()
        if count % 100 == 0:
            print("count = %d, loss = %.5f" %(count, avg_loss / 100))
            save_model(model, 'models2/e' + str(epoch + 1) + 'b' + str(count) + '.pt')
            avg_loss = 0
        del train, label, y, loss
    # print('training loss:', avg_loss / count)
    
    count = 0
    avg_loss = 0
    with torch.no_grad():
        while vloader.has_next():
            train, label = vloader.get_next()
            y = model(train)
            y = y.permute(0, 2, 1)
            loss = criterion(y, label)
            count += 1
            avg_loss += loss.item()
            del train, label, y, loss
    print('validation loss:', avg_loss / count)
    print('epoch finished:', epoch + 1)
    

count = 100, loss = 1.13751
count = 200, loss = 0.90282
count = 300, loss = 0.87040
count = 400, loss = 0.93317
count = 500, loss = 0.87822
count = 600, loss = 0.80013
count = 700, loss = 0.73488
count = 800, loss = 0.71735
count = 900, loss = 0.68227
count = 1000, loss = 0.65094
count = 1100, loss = 0.67153
count = 1200, loss = 0.61712
count = 1300, loss = 0.59793
count = 1400, loss = 0.57281
count = 1500, loss = 0.55068
count = 1600, loss = 0.52966
count = 1700, loss = 0.50706
count = 1800, loss = 0.49406
count = 1900, loss = 0.47140
count = 2000, loss = 0.46135
count = 2100, loss = 0.48029
count = 2200, loss = 0.44616
count = 2300, loss = 0.45807
count = 2400, loss = 0.43263
count = 2500, loss = 0.42324
count = 2600, loss = 0.41414
count = 2700, loss = 0.41976
count = 2800, loss = 0.42209
count = 2900, loss = 0.41599
count = 3000, loss = 0.40216
count = 3100, loss = 0.41158


KeyboardInterrupt: 